Datasets¶
Local and Global Algorithms ...

AQUAINT: Milne

MSNBC dataset, taken from (Cucerzan, 2007),

ACE: Mechanical Turkn

Wiki: choose those paragraphts that p(t|m) makes atleast 10% error

For evaluation, check BOT evaluation, mentioned in Milne

Downloadable from : http://cogcomp.cs.illinois.edu/page/resource_view/4

Spotlight two datasets, a wiki selection 35 paragraphs from New York times There is a website, but couldn't find it

http://oldwiki.dbpedia.org/Datasets/NLP

Tag me: Wiki and tweet, available, but looks old! http://acube.di.unipi.it/tagme-dataset/

AIDA

: https://www.mpi-inf.mpg.de/departments/databases-and-information-systems/research/yago-naga/aida/downloads/** AIDA CoNLL-YAGO Dataset: Hnad create from Conll AIDA-EE Dataset: Again hand done

In [ ]:

# Remove after transferring to wikification module
    if  op_method == 'word2vec_word_context'  :
        return word_context_disambiguate(S, M, C, 5)

# Simple Test

In [ ]:
# %load_ext autoreload
# %autoreload

# %aimport wsd
# import sys
from wikify import *
import time
ws=5
S=["Carlos", "met", "David", "and" , "Victoria", "in", "Madrid"]
M=[[0, "Roberto_Carlos"], [2, "David_Beckham"], [4, "Victoria_Beckham"], [6, "Madrid"]]

S=["David", "met", "Victoria", "and" , "Victoria", "in", "Madrid"]
M=[[0, "David_Beckham"], [2, "Victoria_Beckham"], [6, "Madrid"]]
S=["Phoenix, Arizona"] 
M=[[0, "Phoenix,_Arizona"]]

start = time.time()
C = generate_candidates(S, M, max_t=5, enforce=False)
print "Candidates: ", C, "\n"

ids, titles = wikify(S,M,C, ws, method='context2context')


#print "Key Scores_method_1: ", candslist_scores, "\n"
print "Best IDS", ids, "\n"
print "Best titles", titles, "\n"
#print "get_tp",get_tp(M, ids) 






# WSD Evaluation

In [ ]:
%%writefile wsd_eval.py 
import sys
from optparse import OptionParser

#sys.path.insert(0,'..')
#from wikisim.calcsim import *
from wsd import *
import time
from random import shuffle
np.seterr(all='raise')

# parser = OptionParser()
# parser.add_option("-t", "--max_t", action="store", type="int", dest="max_t", default=5)
# parser.add_option("-c", "--max_count", action="store", type="int", dest="max_count", default=-1)
# parser.add_option("-w", "--win_size", action="store", type="int", dest="win_size", default=5)
# parser.add_option("-v", action="store_true", dest="verbose", default=False)

#(options, args) = parser.parse_args()



# max_t = options.max_t
# max_count = options.max_count
# verbose = options.verbose
# ws = options.win_size


max_t = 20
max_count = -1
verbose = True

fresh_restart=True



dsnames = [os.path.join(home,'backup/datasets/ner/kore.json'),
          os.path.join(home,'backup/datasets/ner/wiki-mentions.5000.json'),
          os.path.join(home,'backup/datasets/ner/msnbc.json'),
          os.path.join(home,'backup/datasets/ner/aquaint.json') 
          ]

methods = ['popularity','keydisamb','entitycontext','mention2entity','context2context','context2profile', 'learned']

outdir = os.path.join(baseresdir, 'wikify')
# if not os.path.exists(outdir): #Causes synchronization problem
#     os.makedirs(outdir)

tmpdir = os.path.join(outdir, 'tmp')
# if not os.path.exists(tmpdir): #Causes synchronization problem
#     os.makedirs(tmpdir)
    
resname =  os.path.join(outdir, 'reslog.csv')
#clearlog(resname)

detailedresname=  os.path.join(outdir, 'detailedreslog.txt')
#clearlog(detailedresname)



for method in methods:
    if 'word2vec' in method:
        gensim_loadmodel(word2vec_path)
        print "loaded"
        sys.stdout.flush()
    for dsname in dsnames:
        start = time.time()
        
        print "dsname: %s, method: %s, max_t: %s ..."  % (dsname,
                method, max_t)
        sys.stdout.flush()
        
        tmpfilename = os.path.join(tmpdir, 
                                   '-'.join([method, str(max_t), os.path.basename(dsname)]))
        overall=[]
        start_count=-1
        if os.path.isfile(tmpfilename) and not fresh_restart:
            with open(tmpfilename,'r') as tmpf:
                for line in tmpf:
                    js = json.loads(line.strip())
                    start_count = js['no']
                    if js['tp'] is not None:
                        overall.append(js['tp'])
        
        if start_count !=-1:
            print "Continuing from\t", start_count
            
        count=0
        with open(dsname,'r') as ds, open(tmpfilename,'a') as tmpf:
            for line in ds:
                js = json.loads(line.decode('utf-8').strip());
                S = js["text"]
                M = js["mentions"]
                count +=1
                if count <= start_count:
                    continue
                if verbose:
                    print "%s:\tS=%s\n\tM=%s" % (count, json.dumps(S, ensure_ascii=False).encode('utf-8'),json.dumps(M, ensure_ascii=False).encode('utf-8'))
                    sys.stdout.flush()
                    
                C = generate_candidates(S, M, max_t=max_t, enforce=False)
                
                try:
                    #ids, titles = disambiguate_driver(S,M, C, ws=0, method=method, direction=direction, op_method=op_method)
                    ids, titles = wsd(S,M,C, method=method)
                    tp = get_tp(ids, M) 
                except Exception as ex:
                    tp = (None, None)
                    print "[Error]:\t", type(ex), ex
                    raise
                    continue
                
                overall.append(tp)
                tmpf.write(json.dumps({"no":count, "tp":tp})+"\n")
                if (max_count !=-1) and (count >= max_count):
                    break
                    

        elapsed = str(timeformat(int(time.time()-start)));
        print "done"
        detailedres ={"dsname":dsname, "method": method, 
                      "max_t": max_t, "tp":overall, "elapsed": elapsed, "problem": "wsd"}
        
        
        logres(detailedresname, '%s',  json.dumps(detailedres))
        
        micro_prec, macro_prec = get_prec(overall)        
        logres(resname, 'wsd\t%s\t%s\t%s\t%s\t%s\t%s', method, max_t, 
               dsname, micro_prec, macro_prec, elapsed)

print "done"

# Wikification Evaluation

In [ ]:
%%writefile wsd_eval.py
"""Evaluating the wsd module. It assumes the sentences are already segmented
"""
import sys
from optparse import OptionParser

#sys.path.insert(0,'..')
#from wikisim.calcsim import *
from wsd import *
import time
from random import shuffle
np.seterr(all='raise')

# parser = OptionParser()
# parser.add_option("-t", "--max_t", action="store", type="int", dest="max_t", default=5)
# parser.add_option("-c", "--max_count", action="store", type="int", dest="max_count", default=-1)
# parser.add_option("-w", "--win_size", action="store", type="int", dest="win_size", default=5)
# parser.add_option("-v", action="store_true", dest="verbose", default=False)

#(options, args) = parser.parse_args()



# max_t = options.max_t
# max_count = options.max_count
# verbose = options.verbose
# ws = options.win_size


max_t = 20
max_count = -1
verbose = True

fresh_restart=True


dsnames = [os.path.join(home,'backup/datasets/ner/kore.json'),
           os.path.join(home,'backup/datasets/ner/wiki-mentions.5000.json'),
          ]

mentionmethods = (CORE_NLP, LEARNED_MENTION)

outdir = os.path.join(baseresdir, 'wikify')
# if not os.path.exists(outdir): #Causes synchronization problem
#     os.makedirs(outdir)

tmpdir = os.path.join(outdir, 'tmp')
# if not os.path.exists(tmpdir): #Causes synchronization problem
#     os.makedirs(tmpdir)
    
resname =  os.path.join(outdir, 'reslog.csv')
#clearlog(resname)

detailedresname=  os.path.join(outdir, 'detailedreslog.txt')
#clearlog(detailedresname)



for mentionmethod in mentionmethods:
    for dsname in dsnames:
        start = time.time()
        
        print "dsname: %s, method: %s, max_t: %s, ws: %s ..."  % (dsname,
                method, max_t, ws)
        sys.stdout.flush()
        
        tmpfilename = os.path.join(tmpdir, 
                                   '-'.join([method, str(max_t), str(ws), os.path.basename(dsname)]))
        overall_tp=[]
        overall_fp=[]
        overall_tn=[]
        start_count=-1
        if os.path.isfile(tmpfilename) and not fresh_restart:
            with open(tmpfilename,'r') as tmpf:
                for line in tmpf:
                    js = json.loads(line.strip())
                    start_count = js['no']
                    if js['tp'] is not None:
                        overall_tp.append(js['tp'])
                        overall_fp.append(js['fp'])
                        overall_tn.append(js['tn'])
        
        if start_count !=-1:
            print "Continuing from\t", start_count
            
        count=0
        with open(dsname,'r') as ds, open(tmpfilename,'a') as tmpf:
            for line in ds:
                js = json.loads(line.decode('utf-8').strip());
                S = js["text"]
                M = js["mentions"]
                count +=1
                if count <= start_count:
                    continue
                if verbose:
                    print "%s:\tS=%s\n\tM=%s" % (count, json.dumps(S, ensure_ascii=False).encode('utf-8'),json.dumps(M, ensure_ascii=False).encode('utf-8'))
                    sys.stdout.flush()
                    
                S2,M2 = wikify_a_line(line, mentionmethod=mentionmethod)
                mention_measures = get_sentence_measures(S2, M2, S, M, wsd_measure=False)
                mention_overall.append(mention_measures)
                
                wikify_measures = get_sentence_measures(S2, M2, S, M, wsd_measure=True)
                wikify_overall.append(wikify_measures)
                
                if (max_count !=-1) and (count >= max_count):
                    break
                    

        elapsed = str(timeformat(int(time.time()-start)));
        print "done"
        detailedres ={"dsname":dsname, "method": method, 
                      "max_t": max_t, "tp":overall, "elapsed": elapsed, "ws": ws}
        
        
        logres(detailedresname, '%s',  json.dumps(detailedres))
        
        mention_overall_measures = get_overall_measures(mention_overall)    
        output = ('mention_evaluation',method, max_t , dsname) + mention_overall_measures + (elapsed,)
        
        logres(resname, '%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n', output)
        
        wikify_overall_measures = get_overall_measures(wikify_overall)  
        output = ('wikify_evaluation',method, max_t , dsname) + wikify_overall_measures + (elapsed,)
            

print "done"